In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.rcParams['font.sans-serif'] = ["Arial"]

In [ ]:
pb_data_A= pd.read_excel('TEP_1.xlsx',sheet_name='model_1')
#pb_data_A.set_index("序号", inplace = True)
#pb_data=np.transpose(peibi_cos_data)
pb_data_A.set_axis(['序号','TEP(ul)','NMP(ul)','2-Me(ul)','Temp(℃)','MACl(%)','conc(mmol/1000ul)','NMP ratio(%)','Average efficiency','The highest efficiency'],axis=1)
#pb_data_A.set_index("序号", inplace = True)
pb_data_A

In [ ]:
df=pb_data_A[['TEP(ul)','NMP(ul)','2-Me(ul)','Temp(℃)','MACl(%)','The highest efficiency']]

df.columns = ['TEP (μL)',  'NMP (μL)', '2-Me (μL)',  'Temp (°C)', 'MACl (%)', 'Efficiency (%)']
df


In [ ]:
import emukit
import GPy
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs.random_design import RandomDesign
from emukit.core.initial_designs.latin_design import LatinDesign

In [ ]:
TEP_min, TEP_max, TEP_step = [500, 651, 10] ## Unit: uL
TEP_var = np.arange(TEP_min, TEP_max+TEP_step*0.1, TEP_step)
TEP_num = len(TEP_var)

NMP_min, NMP_max, NMP_step = [10, 81, 5] ## Unit: uL
NMP_var = np.arange(NMP_min, NMP_max+NMP_step*0.1, NMP_step)
NMP_num = len(NMP_var)

TwoMe_min, TwoMe_max, TwoMe_step = [0, 50, 5] ## Unit: uL
TwoMe_var = np.arange(TwoMe_min, TwoMe_max+TwoMe_step*0.1, TwoMe_step) 
TwoMe_num = len(TwoMe_var)

Temp_min, Temp_max, Temp_step = [120, 150, 5] ## Unit: degree C
Temp_var = np.arange(Temp_min, Temp_max+Temp_step*0.1, Temp_step)
Temp_num = len(Temp_var)

MACl_min, MACl_max, MACl_step = [10, 40, 5] # Unit: %
MACl_var = np.arange(MACl_min, MACl_max+MACl_step*0.1, MACl_step)
MACl_num = len(MACl_var)



var_array = [TEP_var, NMP_var, TwoMe_var, Temp_var, MACl_var]
x_labels = ['TEP (μL)', 
            'NMP (μL)', 
            '2-Me (μL)',  
            'Temp (°C)', 
            'MACl (%)']

In [ ]:
X_all_grid = []
for tep in TEP_var:
    for nmp in NMP_var:
        for twome in TwoMe_var:
            for temp in Temp_var:
                for macl in MACl_var:
                    X_all_grid.append([tep, nmp, twome, temp, macl])
X_all_grid = np.array(X_all_grid)
X_all_grid.shape

In [ ]:
def x_normalizer(X, var_array = var_array):
    
    def max_min_scaler(x, x_max, x_min):
        return (x-x_min)/(x_max-x_min)
    x_norm = []
    for x in (X):
           x_norm.append([max_min_scaler(x[i], 
                         max(var_array[i]), 
                        min(var_array[i])) for i in range(len(x))])
                          #min(var_array[i])) for i in range(len(x))])
    return np.array(x_norm)
def x_denormalizer(x_norm, var_array = var_array):
    
    def max_min_rescaler(x, x_max, x_min):
        return x*(x_max-x_min)+x_min
    x_original = []
    for x in (x_norm):
           x_original.append([max_min_rescaler(x[i], 
                              max(var_array[i]), 
                              #min(var_array[i])) for i in range(len(x))])
                               min(var_array[i])) for i in range(len(x))])
    return np.array(x_original)


def get_closest_array(suggested_x):   
    
    def get_closest_value(given_value, array_list):
        absolute_difference_function = lambda list_value :abs(list_value - given_value)  
        closest_value = min(array_list, key=absolute_difference_function)  
        return closest_value
    
    var_list = var_array
    modified_array = []
    for x in suggested_x:
        modified_array.append([get_closest_value(x[i], var_list[i]) for i in range(len(x))])
    return np.array(modified_array)

In [ ]:
parameter_space = ParameterSpace([ContinuousParameter('TEP', 0-1/(TEP_num-1)/2, 1+1/(TEP_num-1)/2),
                                  ContinuousParameter('NMP', 0-1/(NMP_num-1)/2, 1+1/(NMP_num-1)/2),
                                  ContinuousParameter('TwoMe', 0-1/(TwoMe_num-1)/2, 1+1/(TwoMe_num-1)/2),
                                  ContinuousParameter('Temp', 0-1/(Temp_num-1)/2, 1+1/(Temp_num-1)/2),
                                  ContinuousParameter('MACl', 0-1/(MACl_num-1)/2, 1+1/(MACl_num-1)/2),
                                  
                                  ])

In [ ]:
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
x_exp = x_normalizer(df.iloc[:,0:5].values)
#print(x_exp)
y_exp = np.transpose(([df.iloc[:,-1].values]))
X, Y = [x_exp, y_exp]
#print(Y)
#print(len(x_exp))
input_dim = 5
ker = GPy.kern.Matern52(input_dim = input_dim, ARD = True)#
ker.lengthscale.constrain_bounded(0.01, 10) 
ker.variance.constrain_bounded(1e-2, 10000.0) 
model_gpy = GPRegression(X , -Y, ker)
model_gpy.Gaussian_noise.variance =0.5**2
model_gpy.Gaussian_noise.variance.fix()
model_gpy.randomize()
model_gpy.optimize_restarts(num_restarts=20,verbose =False, messages=False)
objective_model = GPyModelWrapper(model_gpy)
print(objective_model.model.kern.lengthscale)
print(objective_model.model.kern.variance)

In [ ]:
f_obj =  objective_model.model.predict
y_pred, y_uncer = f_obj(X)
y_pred = -y_pred[:,-1]
y_uncer = np.sqrt(y_uncer[:,-1])

from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from scipy.stats import spearmanr
fig, axes = plt.subplots(1, 3, figsize=(5.5*3, 4.5))
fs = 20
lims1 = (0.2, 1.2)
axes[0].scatter(Y[:,-1]/20, y_pred/20, alpha = 0.5, c = 'navy' , edgecolor = 'navy')
axes[0].errorbar(Y[:,-1]/20, y_pred/20, yerr = y_uncer/20, ms = 0, 
                 ls = '', capsize = 2, alpha = 0.6,
                 color = 'gray', zorder = 0)
axes[0].plot(lims1, lims1, 'k--', alpha=0.75, zorder=0)
rmse_value = np.sqrt(mean_squared_error(Y[:,-1], y_pred))
mae_value = mean_absolute_error(Y[:,-1], y_pred)
spearman_value = spearmanr(Y[:,-1], y_pred)[0]
rsquared_value = r2_score(Y[:,-1], y_pred)

print('MAE:',np.round(mae_value,4), ' ',
      'RMSE:', np.round(rmse_value,4), ' ',
      'spearman:', np.round(spearman_value,4), ' ',
      'R² score:', np.round(rsquared_value,4))

title = 'GPR' + " (MAE=%.2f" % mae_value+' (%))'
axes[0].set_xlabel('Truth Normalized PCE (a.u.)', fontproperties='Arial', fontsize=24, labelpad=15)
axes[0].set_ylabel('Prediction Normalized PCE (a.u.)', fontproperties='Arial', fontsize=24, labelpad=15)
axes[0].set_title(title, fontsize = fs,fontproperties='Arial', pad=10)
        
for i in range(len(axes)):
    axes[i].tick_params(direction='in', length=5, width=1, labelsize = fs, grid_alpha = 0.5, pad=10)
    axes[i].grid(True, linestyle='-.')
plt.subplots_adjust(wspace = 0.4)

axes[1].axis("off")
axes[2].axis("off")
plt.savefig("TEP_model1.png", dpi=600, bbox_inches='tight') 
plt.show()

In [ ]:
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement, \
                                                      NegativeLowerConfidenceBound, \
                                                      MaxValueEntropySearch, \
                                                      ProbabilityOfImprovement
from emukit.core.acquisition import IntegratedHyperParameterAcquisition
np.random.seed(10)#to make sure the random results is reproducible 
bs = 8

# ## Expeceted Improvement (EI)
#acquisition = ExpectedImprovement(objective_model, jitter=.1)
# ## Uppper Confidence Bound (UCB)
acquisition = NegativeLowerConfidenceBound(objective_model, beta = 1)
# ## Maximum Value Entropy Search (MES)
#acquisition = MaxValueEntropySearch(objective_model, parameter_space, grid_size = 10000*4)

# Make loop and collect points
bayesopt = BayesianOptimizationLoop(model=objective_model, 
                                   space=parameter_space, 
                                   acquisition=acquisition,
                                   batch_size = bs,
                                   )
                                   #batchsize may need to be >bs due to duplication

print('total condition no.:', len(X))
print('total device no.:', len(bayesopt.loop_state.Y))
print('maximum Y (PCE [%]): ', -np.min(np.transpose(np.round(bayesopt.loop_state.Y, 1))))


f_acq = bayesopt.candidate_point_calculator.acquisition.acquisition.evaluate
f_obj = objective_model.model.predict




acq_all_grid = f_acq(x_normalizer(X_all_grid))
sort_index = np.argsort(acq_all_grid, axis =0)
X_new = []
top = 130*10 # top 1% = 1300/129360 You should set it according to your confidence in the model (balance of exploitation and exploration)
for i in sort_index[-top:]:
        X_new.append(X_all_grid[i][0])
X_new=np.array(X_new)
bs_index = [np.random.randint(top) for i in np.arange(bs)]
X_new = X_new[bs_index]
acq_new= f_acq(x_normalizer(X_new))
y_new_pred, y_new_uncer = f_obj(x_normalizer(X_new))

df_Xnew = pd.DataFrame(X_new, columns = x_labels)
df_all = pd.concat([df.iloc[:,0:5], df_Xnew])
df_all_ = df_all.drop_duplicates()
df_Xnew = df_all_.iloc[len(df):len(df)+bs]
df_Xnew = df_Xnew.sort_values(by=list(df.columns[0:5]), ignore_index = True)
df_Xnew.index = np.arange(len(df_Xnew))+len(df)
print('New X:',len(df_Xnew))
df_Xnew

In [ ]:
device_eff = y_exp
fig, axes = plt.subplots(1, 2, figsize=(16, 4.5), sharey = False)
fs = 20
exp_cond = (np.arange(len(y_exp))+1).reshape(-1,1)
exp_eff = device_eff.reshape(-1,1)

f_obj =  objective_model.model.predict
y_pred, y_uncer = f_obj(X)
y_pred = -y_pred[:,-1]
y_uncer = np.sqrt(y_uncer[:,-1])


axes[0].scatter(exp_cond, exp_eff, #facecolor = 'none',
            edgecolor = 'navy', s = 20, alpha = 0.6, label = 'experiment')

axes[0].plot(exp_cond, np.maximum.accumulate(exp_eff), 
         marker = 'o', ms = 0, c = 'black')

axes[0].scatter(exp_cond, y_pred,
                s = 50, facecolors='none', alpha = 0.6, edgecolor = 'gray', label = 'predicted')
axes[0].errorbar(exp_cond, y_pred, yerr = y_uncer,  
                 ms = 1, ls = '', capsize = 2, alpha = 0.6, 
                 color = 'gray', zorder = 0)


y_pred_new, y_uncer_new = f_obj(x_normalizer(df_Xnew.values))
y_pred_new = -y_pred_new[:,-1]
y_uncer_new = np.sqrt(y_uncer_new[:,-1])

axes[0].scatter(np.arange(len(df_Xnew))+1+len(X), y_pred_new,
                s = 50, facecolors='none', alpha = 0.6, edgecolor = 'darkgreen', label = 'suggested')
axes[0].errorbar(np.arange(len(df_Xnew))+1+len(X), y_pred_new, yerr = y_uncer_new,  
                 ms = 0, ls = '', capsize = 2, alpha = 0.6, 
                 color = 'darkgreen', zorder = 0)


axes[0].set_ylabel('Current Best Efficiency', fontsize = 20)
axes[0].set_xlabel('Process Condition', fontsize = 20)

axes[0].set_ylim(-1, 30)
axes[0].set_xlim(-1, 30)
axes[0].set_xticks(np.arange(0,40,5))
axes[0].legend(fontsize = fs*0.8)

axes[1].axis("off")
for ax in axes:
    ax.tick_params(direction='in', length=5, width=1, labelsize = fs*.8, grid_alpha = 0.5)
    ax.grid(True, linestyle='-.')
plt.subplots_adjust(wspace = 0.4)

plt.show()

In [ ]:
design = RandomDesign(parameter_space)
x_sampled = design.get_samples(200)
x_columns = df_Xnew.iloc[:,0:5].columns
for i in range(input_dim):
    for j in range(input_dim-i-1):
        
## Generate a 2D grid for Contour plot
        ind1 = i
        ind2 = j+i+1
        n_steps =21
        x1x2y_pred, x1x2y_uncer =[[],[]]
        for x1 in np.linspace(0, 1, n_steps):
            for x2 in np.linspace(0, 1, n_steps):
                x_temp = np.copy(x_sampled)
                x_temp[:,ind1] = x1
                x_temp[:,ind2] = x2
                y_pred, y_uncer = f_obj(x_temp)
                x1_org = x_denormalizer(x_temp)[0,ind1]
                x2_org = x_denormalizer(x_temp)[0,ind2]
                y_pred = -y_pred
                x1x2y_pred.append([x1_org, x2_org, np.max(y_pred), np.mean(y_pred), np.min(y_pred)])
                x1x2y_uncer.append([x1_org, x2_org, np.max(np.sqrt(y_uncer)), np.mean(np.sqrt(y_uncer)), np.min(np.sqrt(y_uncer))])
                
        x1 = np.array(x1x2y_pred, dtype=object)[:,0].reshape(n_steps, n_steps)
        x2 = np.array(x1x2y_pred, dtype=object)[:,1].reshape(n_steps, n_steps)
        y_pred_max = np.array(x1x2y_pred, dtype=object)[:,2].reshape(n_steps, n_steps)
        y_pred_mean = np.array(x1x2y_pred, dtype=object)[:,3].reshape(n_steps, n_steps)
        y_pred_min = np.array(x1x2y_pred, dtype=object)[:,4].reshape(n_steps, n_steps)
        
        y_uncer_max = np.array(x1x2y_uncer, dtype=object)[:,2].reshape(n_steps, n_steps)
        y_uncer_mean = np.array(x1x2y_uncer, dtype=object)[:,3].reshape(n_steps, n_steps)
        y_uncer_min = np.array(x1x2y_uncer, dtype=object)[:,4].reshape(n_steps, n_steps)

        fs = 18
        title_pad = 16
        
## Contour for Prediction Efficiency Mean
        fig,axes = plt.subplots(1, 3, figsize=(17, 4), sharey = False, sharex = False)
        colorbar_offset = [20, 14, 4]
        for ax, c_offset, y in zip(axes, colorbar_offset,
                                   [y_pred_max, y_pred_mean, y_pred_min]):#[y_pred_max, y_pred_mean, y_pred_min]
            
            #c_plt1 = ax.contourf(x1, x2, y,levels = np.arange(0,7,0.2)*0.05+c_offset, cmap='plasma', extend = 'both')
            c_plt1 = ax.contourf(x1, x2, y, levels = np.arange(19)*0.5/2+c_offset,
                                 cmap='plasma', extend = 'both')

            # levels = np.arange(19)*0.5+c_offset
            cbar = fig.colorbar(c_plt1, ax= ax)
            cbar.ax.tick_params(labelsize=fs*0.8)
            ax.scatter(x_denormalizer(X)[:, ind1], 
                       x_denormalizer(X)[:, ind2], 
                       s = 40, facecolors='white', alpha = 0.5, edgecolor = 'green')

            ax.scatter(df_Xnew.values[:, ind1], 
                       df_Xnew.values[:, ind2], 
                       s = 50, facecolors='yellow', alpha = 0.9, edgecolor = 'magenta')
            ax.set_xlabel(str(x_columns[ind1]),fontsize =  fs,fontproperties='Arial')
            ax.set_ylabel(str(x_columns[ind2]),fontsize =  fs,fontproperties='Arial')

            x1_delta = (np.max(x1)-np.min(x1))*0.05
            x2_delta = (np.max(x2)-np.min(x2))*0.05
            ax.set_xlim(np.min(x1)-x1_delta, np.max(x1)+x1_delta)
            ax.set_ylim(np.min(x2)-x2_delta, np.max(x2)+x2_delta)
            ax.tick_params(direction='in', length=5, width=1, labelsize = fs*.8)


        axes[0].set_title('objective fcn max', pad = title_pad,fontsize =  fs,fontproperties='Arial')
        axes[1].set_title('objective fcn mean', pad = title_pad,fontsize =  fs,fontproperties='Arial')
        axes[2].set_title('objective fcn min', pad = title_pad,fontsize =  fs,fontproperties='Arial')
        plt.subplots_adjust(wspace = 0.3)
        plt.show()